# Gunter's Space Page GSP Scraper

In [1]:
import bs4
import requests
import pandas as pd
import os

In [4]:
#url = "https://space.skyrocket.de/doc_sdat/globalstar.htm"
oneweblist = [
    "https://space.skyrocket.de/doc_sdat/oneweb.htm"
]

starlinklist = [
    "https://space.skyrocket.de/doc_sdat/starlink-v0-9.htm",
    "https://space.skyrocket.de/doc_sdat/starlink-v1-0.htm",
    "https://space.skyrocket.de/doc_sdat/starlink-v1-5.htm",
    "https://space.skyrocket.de/doc_sdat/starlink-v2-mini.htm"
]

globalstarlist = [
    "https://space.skyrocket.de/doc_sdat/globalstar.htm",
    "https://space.skyrocket.de/doc_sdat/globalstar-2.htm"
]

iridiumlist = [
    "https://space.skyrocket.de/doc_sdat/iridium.htm",
    "https://space.skyrocket.de/doc_sdat/iridium-next.htm"
]

orbcommlist = [
    "https://space.skyrocket.de/doc_sdat/orbcomm.htm",
    "https://space.skyrocket.de/doc_sdat/orbcomm-ql.htm",
    "https://space.skyrocket.de/doc_sdat/orbcomm-2.htm"
]

swarmlist = [
    "https://space.skyrocket.de/doc_sdat/spacebee.htm",
    "https://space.skyrocket.de/doc_sdat/spacebee-5.htm",
    "https://space.skyrocket.de/doc_sdat/spacebee-10.htm"
]

urllist = starlinklist

In [5]:
def scrape_url(url):
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text)

    satdata = soup.find("table", {"id": "satdata"})
    satlist = soup.find("table", {"id": "satlist"})

    def parse_row(trrow):
        tdlist = trrow.find_all("td")
        namelist = [td.text for td in tdlist]
        return namelist

    satlist_dict = {
        "Satellite": [],
        "COSPAR": [],
        "Date": [],
        "LS": [],
        "Failed": [],
        "Launche Vehicle": [],
        "Remarks": []
    }

    for trrow in satlist.find_all("tr")[1:]:
        #if
        namelist = parse_row(trrow)
        datestr = namelist[2].lower()
        if(datestr == "cancelled" or datestr == "2023" or datestr == "not launched"):
            continue
        try:
            date = pd.to_datetime(datestr, format="%d.%m.%Y")
        except:
            continue
        satlist_dict["Satellite"].append(namelist[0])
        satlist_dict["COSPAR"].append(namelist[1])
        satlist_dict["Date"].append(date)
        satlist_dict["LS"].append(namelist[3])
        satlist_dict["Failed"].append(namelist[4])
        satlist_dict["Launche Vehicle"].append(namelist[5])
        satlist_dict["Remarks"].append(namelist[6])

    df = pd.DataFrame(satlist_dict)
    return df

dflist = [scrape_url(url) for url in urllist]
df = pd.concat(dflist)

In [6]:
# Checking launchdates
print("Count", len(df.groupby(by="Date").count()))
print(df.groupby(by="Date").count().index)

Count 79
DatetimeIndex(['2019-05-24', '2019-11-11', '2020-01-07', '2020-01-29',
               '2020-02-17', '2020-03-18', '2020-04-22', '2020-06-04',
               '2020-06-13', '2020-08-07', '2020-08-18', '2020-09-03',
               '2020-10-06', '2020-10-18', '2020-10-24', '2020-11-24',
               '2021-01-20', '2021-01-24', '2021-02-04', '2021-02-16',
               '2021-03-04', '2021-03-11', '2021-03-14', '2021-03-24',
               '2021-04-07', '2021-04-29', '2021-05-04', '2021-05-09',
               '2021-05-15', '2021-05-26', '2021-06-30', '2021-09-14',
               '2021-11-13', '2021-12-02', '2021-12-18', '2022-01-06',
               '2022-01-19', '2022-02-03', '2022-02-21', '2022-02-25',
               '2022-03-03', '2022-03-09', '2022-03-19', '2022-04-21',
               '2022-04-29', '2022-05-06', '2022-05-13', '2022-05-14',
               '2022-05-18', '2022-06-17', '2022-07-07', '2022-07-11',
               '2022-07-17', '2022-07-22', '2022-07-24', '2022-08-10